In [47]:
import random
from environment import Agent, Environment
from planner import RoutePlanner
from simulator import Simulator
import pandas as pd
import numpy as np

class LearningAgent(Agent):
    """An agent that learns to drive in the smartcab world."""

    def __init__(self, env):
        super(LearningAgent, self).__init__(env)  # sets self.env = env, state = None, next_waypoint = None, and a default color
        self.color = 'red'  # override color
        self.planner = RoutePlanner(self.env, self)  # simple route planner to get next_waypoint
        # TODO: Initialize any additional variables here
        
        #Set traffic_light and movement
        traffic_light=["red","green"]
        motion = [None, 'forward', 'left', 'right']
        waypoint,oncoming,left=motion,motion,motion

        
        #Initialize q_table
        #Set all features to 0
        self.q_table = {}
        for light in traffic_light:
            for point in waypoint:
                for on in oncoming:
                    for lf in left:
                        self.q_table[(light, point, on, lf)] = {None: 0, 'forward': 0, 'left': 0, 'right': 0}

        print self.q_table


        self.episode=0
        self.preserve=[]

    def reset(self,destination=None,total=0):

        self.planner.route_to(destination)
        # TODO: Prepare for a new trip; reset any variables here, if required
        self.total_reward=total

        
    def update(self, t):
        # Gather inputs
        self.next_waypoint = self.planner.next_waypoint()  # from route planner, also displayed by simulator
        inputs = self.env.sense(self)
        deadline = self.env.get_deadline(self)
        
        
        old_time=t
        print 'Old_TIME',old_time

       



        # TODO: Update state
        self.state = (inputs['light'],
                      self.next_waypoint,
                      inputs['oncoming'],
                      inputs['left'])

        print "The current state is: {}".format(self.state)

        print "t:{}".format(t)
        
        # TODO: Select action according to your policy
        epsilon=0.1
        rand=random.random()
        if rand<epsilon:
            action=random.choice(Environment.valid_actions[0:])
            print "RANDOM ACTION"
        else:
            if max(self.q_table[self.state].values())==0:
                action=random.choice(Environment.valid_actions[0:])
                print self.q_table[self.state]
               
            else:
                action = max(self.q_table[self.state],
                     key=self.q_table[self.state].get)

            print action

        # Execute action and get reward
        reward = self.env.act(self, action)
        
        print "REWARD IS:",reward
        self.total_reward+=reward
     
        print "Total Reward",self.total_reward
        
        # TODO: Learn policy based on state, action, reward
        # Set the tuning parameters
        alpha = 1.0/(1.0+t) # learning rate
        gamma = 1.0/(1.0+deadline) # discount factor
      #  alpha=0.5
       # gamma=0.2
        # Get the new state after the above action

        inputs_new = self.env.sense(self)
        state_new = (inputs_new['light'],
                     self.planner.next_waypoint(),
                     inputs_new['oncoming'],
                     inputs_new['left'])
        print "The new state is: {}".format(state_new)
        print "t:{}".format(t)
        print alpha,gamma
        



        
        # Calculate the Q_value
        q_value = (1 - alpha) * self.q_table[self.state][action] + \
                  alpha * (reward + gamma * max(self.q_table[state_new].values()))
        # Update the Q_table
        self.q_table[self.state][action] = q_value
        # Set current state and action as previous state and action
        
        
        print "LearningAgent.update(): deadline = {}, inputs = {}, action = {}, reward = {}".format(deadline, inputs, action, reward)  # [debug]
        new_time=t
        print 'New_time',new_time
        total_preserve=self.total_reward
        print "\n"
        print "\n"



        if old_time==0:
            self.episode+=1

        self.preserve.append([new_time,self.total_reward,deadline,self.episode])
      #  self.preserve.append(new_time)
      #  self.preserve.append(self.total_reward)
      #  self.preserve.append(self.episode)


        
        
        
       
        
        if self.episode==10:
            df1=pd.DataFrame(self.preserve,columns=['Time','Reward','Deadline','Episode'])
            print self.preserve
            print df1
            return df1
       
        
    
def run():
    """Run the agent for a finite number of trials."""
    # Set up environment and agent
    e = Environment()  # create environment (also adds some dummy traffic)
    a = e.create_agent(LearningAgent)  # create agent
    e.set_primary_agent(a, enforce_deadline=True)  # set agent to track
    # Now simulate it
    sim = Simulator(e, update_delay=0.1,display=True)  # reduce update_delay to speed up simulation
    sim.run(n_trials=10)  # press Esc or close pygame window to quit




    

if __name__ == '__main__':
    run()
    
    


{('red', None, 'left', 'right'): {'forward': 0, 'right': 0, None: 0, 'left': 0}, ('green', None, 'right', 'right'): {'forward': 0, 'right': 0, None: 0, 'left': 0}, ('green', 'left', None, 'forward'): {'forward': 0, 'right': 0, None: 0, 'left': 0}, ('green', 'left', 'left', None): {'forward': 0, 'right': 0, None: 0, 'left': 0}, ('green', None, 'right', 'left'): {'forward': 0, 'right': 0, None: 0, 'left': 0}, ('red', 'right', 'left', None): {'forward': 0, 'right': 0, None: 0, 'left': 0}, ('green', 'left', 'right', 'forward'): {'forward': 0, 'right': 0, None: 0, 'left': 0}, ('red', 'left', 'right', 'forward'): {'forward': 0, 'right': 0, None: 0, 'left': 0}, ('red', 'left', 'left', 'left'): {'forward': 0, 'right': 0, None: 0, 'left': 0}, ('green', 'forward', 'right', 'right'): {'forward': 0, 'right': 0, None: 0, 'left': 0}, ('red', 'left', None, 'right'): {'forward': 0, 'right': 0, None: 0, 'left': 0}, ('green', 'forward', 'forward', None): {'forward': 0, 'right': 0, None: 0, 'left': 0}, (

In [2]:
a=[[0, 2.0, 30, 1], [1, 1.5, 29, 1], [2, 3.5, 28, 1], [3, 3.0, 27, 1], [4, 3.0, 26, 1], [5, 5.0, 25, 1], [6, 5.0, 24, 1], [7, 5.0, 23, 1], [8, 4.5, 22, 1], [9, 4.0, 21, 1], [10, 6.0, 20, 1], [11, 8.0, 19, 1], [12, 7.5, 18, 1], [13, 9.5, 17, 1], [14, 8.5, 16, 1], [15, 8.0, 15, 1], [16, 10.0, 14, 1], [17, 12.0, 13, 1], [18, 14.0, 12, 1], [19, 14.0, 11, 1], [20, 13.5, 10, 1], [21, 13.5, 9, 1], [22, 13.5, 8, 1], [23, 15.5, 7, 1], [24, 17.5, 6, 1], [25, 19.5, 5, 1], [26, 18.5, 4, 1], [27, 17.5, 3, 1], [28, 16.5, 2, 1], [29, 16.0, 1, 1], [30, 18.0, 0, 1], [0, 2.0, 40, 2], [1, 4.0, 39, 2], [2, 3.0, 38, 2], [3, 3.0, 37, 2], [4, 5.0, 36, 2], [5, 14.5, 35, 2], [0, -1.0, 25, 3], [1, -2.0, 24, 3], [2, -3.0, 23, 3], [3, -1.0, 22, 3], [4, -2.0, 21, 3], [5, -2.0, 20, 3], [6, -2.5, 19, 3], [7, -3.0, 18, 3], [8, -1.0, 17, 3], [9, 1.0, 16, 3], [10, 0.5, 15, 3], [11, 2.5, 14, 3], [12, 2.0, 13, 3], [13, 4.0, 12, 3], [14, 6.0, 11, 3], [15, 8.0, 10, 3], [16, 10.0, 9, 3], [17, 12.0, 8, 3], [18, 11.5, 7, 3], [19, 13.5, 6, 3], [20, 13.5, 5, 3], [21, 12.5, 4, 3], [22, 14.5, 3, 3], [23, 16.5, 2, 3], [24, 15.5, 1, 3], [25, 14.5, 0, 3], [0, 2.0, 35, 4], [1, 2.0, 34, 4], [2, 1.5, 33, 4], [3, 3.5, 32, 4], [4, 3.0, 31, 4], [5, 5.0, 30, 4], [6, 7.0, 29, 4], [7, 7.0, 28, 4], [8, 7.0, 27, 4], [9, 6.0, 26, 4], [10, 18.0, 25, 4], [0, -0.5, 30, 5], [1, 1.5, 29, 5], [2, 3.5, 28, 5], [3, 3.0, 27, 5], [4, 2.5, 26, 5], [5, 4.5, 25, 5], [6, 6.5, 24, 5], [7, 6.0, 23, 5], [8, 8.0, 22, 5], [9, 10.0, 21, 5], [10, 12.0, 20, 5], [11, 11.0, 19, 5], [12, 10.0, 18, 5], [13, 9.5, 17, 5], [14, 8.5, 16, 5], [15, 8.5, 15, 5], [16, 10.5, 14, 5], [17, 9.5, 13, 5], [18, 8.5, 12, 5], [19, 8.5, 11, 5], [20, 20.5, 10, 5], [0, 2.0, 20, 6], [1, 4.0, 19, 6], [2, 3.5, 18, 6], [3, 5.5, 17, 6], [4, 7.5, 16, 6], [5, 9.5, 15, 6], [6, 11.5, 14, 6], [7, 23.5, 13, 6], [0, 2.0, 20, 7], [1, 2.0, 19, 7], [2, 1.0, 18, 7], [3, 0.0, 17, 7], [4, 2.0, 16, 7], [5, 2.0, 15, 7], [6, 2.0, 14, 7], [7, 2.0, 13, 7], [8, 4.0, 12, 7], [9, 4.0, 11, 7], [10, 3.0, 10, 7], [11, 2.5, 9, 7], [12, 2.0, 8, 7], [13, 4.0, 7, 7], [14, 6.0, 6, 7], [15, 18.0, 5, 7], [0, -0.5, 25, 8], [1, -1.5, 24, 8], [2, -2.5, 23, 8], [3, -3.5, 22, 8], [4, -3.5, 21, 8], [5, -1.5, 20, 8], [6, -2.5, 19, 8], [7, -2.5, 18, 8], [8, -2.5, 17, 8], [9, -3.5, 16, 8], [10, -1.5, 15, 8], [11, -2.5, 14, 8], [12, -0.5, 13, 8], [13, -0.5, 12, 8], [14, -1.5, 11, 8], [15, 10.5, 10, 8], [0, 2.0, 20, 9], [1, 1.0, 19, 9], [2, 0.5, 18, 9], [3, 0.0, 17, 9], [4, -0.5, 16, 9], [5, -1.5, 15, 9], [6, -2.5, 14, 9], [7, -3.5, 13, 9], [8, -4.5, 12, 9], [9, -5.0, 11, 9], [10, -5.0, 10, 9], [11, -3.0, 9, 9], [12, -1.0, 8, 9], [13, -2.0, 7, 9], [14, -3.0, 6, 9], [15, -4.0, 5, 9], [16, -2.0, 4, 9], [17, 0.0, 3, 9], [18, -1.0, 2, 9], [19, -2.0, 1, 9], [20, -2.0, 0, 9], [0, 2.0, 25, 10], [1, 4.0, 24, 10], [2, 6.0, 23, 10], [3, 8.0, 22, 10], [4, 7.5, 21, 10], [5, 9.5, 20, 10], [6, 11.5, 19, 10], [7, 13.5, 18, 10], [8, 12.5, 17, 10], [9, 12.5, 16, 10], [10, 12.0, 15, 10], [11, 14.0, 14, 10], [12, 16.0, 13, 10], [13, 18.0, 12, 10], [14, 17.5, 11, 10], [15, 17.0, 10, 10], [16, 19.0, 9, 10], [17, 18.0, 8, 10], [18, 20.0, 7, 10], [19, 22.0, 6, 10], [20, 21.0, 5, 10], [21, 20.0, 4, 10], [22, 19.0, 3, 10], [23, 19.0, 2, 10], [24, 19.0, 1, 10], [25, 18.5, 0, 10]]

In [3]:
b=[[0, 2.0, 30, 1], [1, 1.5, 29, 1], [2, 3.5, 28, 1], [3, 3.0, 27, 1], [4, 3.0, 26, 1], [5, 5.0, 25, 1], [6, 5.0, 24, 1], [7, 5.0, 23, 1], [8, 4.5, 22, 1], [9, 4.0, 21, 1], [10, 6.0, 20, 1], [11, 8.0, 19, 1], [12, 7.5, 18, 1], [13, 9.5, 17, 1], [14, 8.5, 16, 1], [15, 8.0, 15, 1], [16, 10.0, 14, 1], [17, 12.0, 13, 1], [18, 14.0, 12, 1], [19, 14.0, 11, 1], [20, 13.5, 10, 1], [21, 13.5, 9, 1], [22, 13.5, 8, 1], [23, 15.5, 7, 1], [24, 17.5, 6, 1], [25, 19.5, 5, 1], [26, 18.5, 4, 1], [27, 17.5, 3, 1], [28, 16.5, 2, 1], [29, 16.0, 1, 1], [30, 18.0, 0, 1], [0, 2.0, 40, 2], [1, 4.0, 39, 2], [2, 3.0, 38, 2], [3, 3.0, 37, 2], [4, 5.0, 36, 2], [5, 14.5, 35, 2], [0, -1.0, 25, 3], [1, -2.0, 24, 3], [2, -3.0, 23, 3], [3, -1.0, 22, 3], [4, -2.0, 21, 3], [5, -2.0, 20, 3], [6, -2.5, 19, 3], [7, -3.0, 18, 3], [8, -1.0, 17, 3], [9, 1.0, 16, 3], [10, 0.5, 15, 3], [11, 2.5, 14, 3], [12, 2.0, 13, 3], [13, 4.0, 12, 3], [14, 6.0, 11, 3], [15, 8.0, 10, 3], [16, 10.0, 9, 3], [17, 12.0, 8, 3], [18, 11.5, 7, 3], [19, 13.5, 6, 3], [20, 13.5, 5, 3], [21, 12.5, 4, 3], [22, 14.5, 3, 3], [23, 16.5, 2, 3], [24, 15.5, 1, 3], [25, 14.5, 0, 3], [0, 2.0, 35, 4], [1, 2.0, 34, 4], [2, 1.5, 33, 4], [3, 3.5, 32, 4], [4, 3.0, 31, 4], [5, 5.0, 30, 4], [6, 7.0, 29, 4], [7, 7.0, 28, 4], [8, 7.0, 27, 4], [9, 6.0, 26, 4], [10, 18.0, 25, 4], [0, -0.5, 30, 5], [1, 1.5, 29, 5], [2, 3.5, 28, 5], [3, 3.0, 27, 5], [4, 2.5, 26, 5], [5, 4.5, 25, 5], [6, 6.5, 24, 5], [7, 6.0, 23, 5], [8, 8.0, 22, 5], [9, 10.0, 21, 5], [10, 12.0, 20, 5], [11, 11.0, 19, 5], [12, 10.0, 18, 5], [13, 9.5, 17, 5], [14, 8.5, 16, 5], [15, 8.5, 15, 5], [16, 10.5, 14, 5], [17, 9.5, 13, 5], [18, 8.5, 12, 5], [19, 8.5, 11, 5], [20, 20.5, 10, 5], [0, 2.0, 20, 6], [1, 4.0, 19, 6], [2, 3.5, 18, 6], [3, 5.5, 17, 6], [4, 7.5, 16, 6], [5, 9.5, 15, 6], [6, 11.5, 14, 6], [7, 23.5, 13, 6], [0, 2.0, 20, 7], [1, 2.0, 19, 7], [2, 1.0, 18, 7], [3, 0.0, 17, 7], [4, 2.0, 16, 7], [5, 2.0, 15, 7], [6, 2.0, 14, 7], [7, 2.0, 13, 7], [8, 4.0, 12, 7], [9, 4.0, 11, 7], [10, 3.0, 10, 7], [11, 2.5, 9, 7], [12, 2.0, 8, 7], [13, 4.0, 7, 7], [14, 6.0, 6, 7], [15, 18.0, 5, 7], [0, -0.5, 25, 8], [1, -1.5, 24, 8], [2, -2.5, 23, 8], [3, -3.5, 22, 8], [4, -3.5, 21, 8], [5, -1.5, 20, 8], [6, -2.5, 19, 8], [7, -2.5, 18, 8], [8, -2.5, 17, 8], [9, -3.5, 16, 8], [10, -1.5, 15, 8], [11, -2.5, 14, 8], [12, -0.5, 13, 8], [13, -0.5, 12, 8], [14, -1.5, 11, 8], [15, 10.5, 10, 8], [0, 2.0, 20, 9], [1, 1.0, 19, 9], [2, 0.5, 18, 9], [3, 0.0, 17, 9], [4, -0.5, 16, 9], [5, -1.5, 15, 9], [6, -2.5, 14, 9], [7, -3.5, 13, 9], [8, -4.5, 12, 9], [9, -5.0, 11, 9], [10, -5.0, 10, 9], [11, -3.0, 9, 9], [12, -1.0, 8, 9], [13, -2.0, 7, 9], [14, -3.0, 6, 9], [15, -4.0, 5, 9], [16, -2.0, 4, 9], [17, 0.0, 3, 9], [18, -1.0, 2, 9], [19, -2.0, 1, 9], [20, -2.0, 0, 9], [0, 2.0, 25, 10], [1, 4.0, 24, 10], [2, 6.0, 23, 10], [3, 8.0, 22, 10], [4, 7.5, 21, 10], [5, 9.5, 20, 10], [6, 11.5, 19, 10], [7, 13.5, 18, 10], [8, 12.5, 17, 10], [9, 12.5, 16, 10], [10, 12.0, 15, 10], [11, 14.0, 14, 10], [12, 16.0, 13, 10], [13, 18.0, 12, 10], [14, 17.5, 11, 10], [15, 17.0, 10, 10], [16, 19.0, 9, 10], [17, 18.0, 8, 10], [18, 20.0, 7, 10], [19, 22.0, 6, 10], [20, 21.0, 5, 10], [21, 20.0, 4, 10], [22, 19.0, 3, 10], [23, 19.0, 2, 10], [24, 19.0, 1, 10]]

In [6]:
import pandas as pd
df=pd.DataFrame(b,columns=['Time','Reward','Deadline','Episode'])

In [8]:
df1

NameError: name 'df1' is not defined

In [51]:
print LearningAgent.update.deadline

AttributeError: 'function' object has no attribute 'deadline'